In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 8.4 MB/s eta 0:00:00


Seeing if we are able to use GPU or not

In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


<h2>RANKING USING all-MiniLM-L6-v2 MODEL</h2>

Here, basically we are loading the pre-trained model all-MiniLM-L6-v2 model.<br>Here, we are now loading the json files of documents and queries. Then without any pre-processing, we feed the data to the model and get the encodings for both of them.<br>Further, we used cosine similarity to calculate the similarity between each pair of documents and queries. Then we sorted the results for each query, which gives the top ranked documents for each query. Further we only display top 5 ranked documents for every query.

In [3]:
# Import necessary libraries
import json
from sentence_transformers import SentenceTransformer, util
import torch

# Load the model on the GPU
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

# Load queries and documents from JSON files
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Paths to your JSON files
query_file_path = '/kaggle/input/data-set/Assignment 2/queries.json'  # Replace with your query JSON file path
document_file_path = '/kaggle/input/data-set/Assignment 2/documents.json'  # Replace with your document JSON file path

# Load queries and documents
queries = load_json(query_file_path)
documents = load_json(document_file_path)

# Encode queries and documents on the GPU
query_embeddings = model.encode(queries, convert_to_tensor=True, device='cuda')
document_embeddings = model.encode(documents, convert_to_tensor=True, device='cuda')

# Calculate similarities between queries and documents (remains on GPU)
similarity_scores = util.cos_sim(query_embeddings, document_embeddings)

# Rank documents for each query based on similarity, keeping all operations on the GPU
ranked_results = {}
for idx, query in enumerate(queries):
    # Extract similarities for the current query
    scores = similarity_scores[idx]
    # Sort document indices by similarity score in descending order
    ranked_indices = torch.argsort(scores, descending=True)
    # Prepare a ranked list of document indices and their scores for this query (top 5)
    ranked_documents = [(i, scores[i].item()) for i in ranked_indices[:5]]  # Top 5 only
    ranked_results[idx] = ranked_documents

# Output only the top 5 ranked results for each query, displaying indices
for query_idx, ranked_docs in ranked_results.items():
    print(f"Query Index: {query_idx}")
    for rank, (doc_idx, score) in enumerate(ranked_docs, start=1):
        print(f"{rank}: Document Index: {doc_idx} (Score: {score:.4f})")
    print("\n")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Batches:   0%|          | 0/202 [00:00<?, ?it/s]

Query Index: 0
1: Document Index: 416 (Score: 0.7837)
2: Document Index: 2387 (Score: 0.7376)
3: Document Index: 2388 (Score: 0.7018)
4: Document Index: 2766 (Score: 0.6931)
5: Document Index: 4664 (Score: 0.6820)


Query Index: 1
1: Document Index: 971 (Score: 0.8399)
2: Document Index: 969 (Score: 0.8007)
3: Document Index: 2235 (Score: 0.7606)
4: Document Index: 4858 (Score: 0.7128)
5: Document Index: 970 (Score: 0.7112)


Query Index: 2
1: Document Index: 1767 (Score: 0.8080)
2: Document Index: 1768 (Score: 0.7925)
3: Document Index: 66 (Score: 0.7857)
4: Document Index: 5072 (Score: 0.7652)
5: Document Index: 67 (Score: 0.7226)


Query Index: 3
1: Document Index: 822 (Score: 0.8223)
2: Document Index: 820 (Score: 0.8133)
3: Document Index: 331 (Score: 0.7894)
4: Document Index: 330 (Score: 0.7855)
5: Document Index: 4047 (Score: 0.7503)


Query Index: 4
1: Document Index: 2032 (Score: 0.7202)
2: Document Index: 1543 (Score: 0.6981)
3: Document Index: 2784 (Score: 0.6941)
4: Docume

Basic Pre-Processing of the words is done, where we first tokenize the data and then make 2 copies for each document and query. <br>For one copy, we removed the stop words, while the other one remains intact. Then we apply Porter Stemming to both of them.

In [4]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download NLTK data (run this only if needed)
nltk.download('punkt')
nltk.download('stopwords')

# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Tokenize and preprocess the text data
def preprocess_text(data, remove_stopwords=False):
    processed_data = []
    for item in data:
        # Tokenize the text
        tokens = word_tokenize(item)
        
        # Optionally remove stop words
        if remove_stopwords:
            tokens = [word for word in tokens if word.lower() not in stop_words]
        
        # Apply Porter Stemming
        stemmed_tokens = [stemmer.stem(word) for word in tokens]
        
        # Reconstruct the processed text and add it to the list
        processed_data.append(" ".join(stemmed_tokens))
    
    return processed_data

# Preprocess the data
# Queries
queries_tokenized = preprocess_text(queries, remove_stopwords=False)
queries_tokenized_no_stopwords = preprocess_text(queries, remove_stopwords=True)

# Documents
documents_tokenized = preprocess_text(documents, remove_stopwords=False)
documents_tokenized_no_stopwords = preprocess_text(documents, remove_stopwords=True)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
print(queries_tokenized[0])
print(queries_tokenized_no_stopwords[0])
print(documents_tokenized[0])
print(documents_tokenized_no_stopwords[0])

what Is the cost Of disabl insur for individu ?
cost disabl insur individu ?
the most import step is to find an insur agent who will listen to you , provid sound financi advic and commun clearli with you and the insur compani . pictur are invalu so photograph everyth you own . On a piec of paper list the item in the photo and what it would cost to replac them . that set your person properti limit . then select as much liabil insur as the compani offer . A lawsuit can ruin you financi .
import step find insur agent listen , provid sound financi advic commun clearli insur compani . pictur invalu photograph everyth . piec paper list item photo would cost replac . set person properti limit . select much liabil insur compani offer . lawsuit ruin financi .


Now, calculating the TF and TF-IDF matrix for both the document representations where one doesn't have stop words, while the other one has them.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Calculate TF and TF-IDF
def calculate_tf_tfidf(documents):
    # Initialize CountVectorizer and TfidfVectorizer
    count_vectorizer = CountVectorizer()
    tfidf_vectorizer = TfidfVectorizer()

    # Calculate Term Frequency (TF)
    tf_matrix = count_vectorizer.fit_transform(documents)

    # Calculate TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    return tf_matrix, tfidf_matrix

# Calculate TF and TF-IDF for documents with stop words
tf_with_stopwords, tfidf_with_stopwords = calculate_tf_tfidf(documents_tokenized)

# Calculate TF and TF-IDF for documents without stop words
tf_no_stopwords, tfidf_no_stopwords = calculate_tf_tfidf(documents_tokenized_no_stopwords)

In [16]:
print(tf_with_stopwords.shape)
print(tf_no_stopwords.shape)
print(tfidf_with_stopwords.shape)
print(tfidf_no_stopwords.shape)

(6462, 7997)
(6462, 7947)
(6462, 7997)
(6462, 7947)


<h2>LATENT SEMANTIC INDEXING(LSI) MODEL</h2><br>
Here, we are taking top 100 stems across the entire corpus of documents and then further representing the documents and queries in that space.<br>
Further, we apply SVD to reduce both of them in the CONCEPT space. Further, we used cosine similarity to calculate the similarity between the documents and the queries.

In [19]:
import numpy as np

# Step 1: Construct the TF-IDF matrix for the entire document corpus
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_documents = tfidf_vectorizer.fit_transform(documents_tokenized)

# Step 2: Select the top p stems based on their global importance (sum of TF-IDF scores across all documents)
p = 100  # Replace with desired value of p
tfidf_scores = tfidf_matrix_documents.sum(axis=0).A1  # Sum TF-IDF scores for each term across all documents
top_p_indices = np.argsort(tfidf_scores)[-p:]         # Get indices of top p terms

# Step 3: Create a vocabulary of the top p stems
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
top_p_stems = feature_names[top_p_indices]

# Step 4: Build new TF-IDF matrices for documents and queries using the top p stems only
# Initialize TfidfVectorizer with the reduced vocabulary (top p stems)
tfidf_vectorizer_top_p = TfidfVectorizer(vocabulary=top_p_stems)

# Construct the reduced TF-IDF matrix for documents
tfidf_matrix_documents_top_p = tfidf_vectorizer_top_p.fit_transform(documents)

# Construct the reduced TF-IDF matrix for queries using the same top p vocabulary
tfidf_matrix_queries_top_p = tfidf_vectorizer_top_p.transform(queries)

# Output: TF-IDF matrices for documents and queries using only the top p stems
print("Reduced TF-IDF matrix for documents:", tfidf_matrix_documents_top_p)
print("Reduced TF-IDF matrix for queries:", tfidf_matrix_queries_top_p)

Reduced TF-IDF matrix for documents: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 169214 stored elements and shape (6462, 100)>
  Coords	Values
  (0, 99)	0.39174859937894024
  (0, 98)	0.3823861945312765
  (0, 97)	0.24773999809420458
  (0, 95)	0.0898387657630183
  (0, 94)	0.08899516090679921
  (0, 91)	0.11037906266288217
  (0, 90)	0.2742745865808222
  (0, 89)	0.10063141375818944
  (0, 87)	0.1062213897937984
  (0, 85)	0.1204057050625347
  (0, 84)	0.11438730893984353
  (0, 81)	0.1265096665413472
  (0, 80)	0.1214107260793785
  (0, 77)	0.2843661949414387
  (0, 76)	0.13325467209327618
  (0, 73)	0.13493683902643547
  (0, 71)	0.1417585165618864
  (0, 64)	0.1721276269713557
  (0, 52)	0.19953560354397892
  (0, 45)	0.1915126017887612
  (0, 39)	0.18249601388935904
  (0, 29)	0.22418863822706414
  (0, 25)	0.1910759251938143
  (0, 8)	0.212321229365282
  (0, 2)	0.218176777511187
  :	:
  (6461, 87)	0.20856394775245202
  (6461, 86)	0.036956396893646784
  (6461, 85)	0.11820730847388038
 

In [20]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Assume tfidf_matrix_documents_top_p and tfidf_matrix_queries_top_p have been computed from the previous code
# These matrices are in the same feature space with the top p stems vocabulary

# Parameters
num_topics = 100  # Number of dimensions to reduce to with LSI

# Step 1: Apply LSI using Truncated SVD on the document TF-IDF matrix
svd = TruncatedSVD(n_components=num_topics)
lsi_document_matrix = svd.fit_transform(tfidf_matrix_documents_top_p)

# Step 2: Transform the query TF-IDF matrix to the same LSI space
lsi_query_matrix = svd.transform(tfidf_matrix_queries_top_p)

# Step 3: Calculate cosine similarity between each query and each document in the LSI-reduced space
similarity_matrix = cosine_similarity(lsi_query_matrix, lsi_document_matrix)

# Step 4: Rank documents for each query based on similarity and display top 5 results
top_k = 5
for query_idx, similarities in enumerate(similarity_matrix):
    # Get indices of top 5 most similar documents
    top_doc_indices = np.argsort(similarities)[-top_k:][::-1]
    print(f"Query {query_idx + 1}:")
    for rank, doc_idx in enumerate(top_doc_indices, start=1):
        print(f"  Rank {rank}: Document {doc_idx + 1} (Score: {similarities[doc_idx]:.4f})")
    print("\n")

Query 1:
  Rank 1: Document 3574 (Score: 0.7440)
  Rank 2: Document 3277 (Score: 0.6685)
  Rank 3: Document 4265 (Score: 0.6443)
  Rank 4: Document 2750 (Score: 0.6201)
  Rank 5: Document 5947 (Score: 0.6068)


Query 2:
  Rank 1: Document 1830 (Score: 0.8261)
  Rank 2: Document 5170 (Score: 0.8261)
  Rank 3: Document 1730 (Score: 0.6793)
  Rank 4: Document 5159 (Score: 0.6316)
  Rank 5: Document 5468 (Score: 0.6305)


Query 3:
  Rank 1: Document 6283 (Score: 0.6869)
  Rank 2: Document 1874 (Score: 0.6249)
  Rank 3: Document 1767 (Score: 0.6159)
  Rank 4: Document 1619 (Score: 0.5962)
  Rank 5: Document 1426 (Score: 0.5863)


Query 4:
  Rank 1: Document 1600 (Score: 0.8049)
  Rank 2: Document 244 (Score: 0.5704)
  Rank 3: Document 4828 (Score: 0.5431)
  Rank 4: Document 961 (Score: 0.5400)
  Rank 5: Document 5667 (Score: 0.5353)


Query 5:
  Rank 1: Document 5846 (Score: 0.6371)
  Rank 2: Document 763 (Score: 0.6351)
  Rank 3: Document 2549 (Score: 0.6349)
  Rank 4: Document 4967 (Score

Further, here we used the same steps but now, we are using documents and queries after removing stop words and then calculating the similarity between them.

In [21]:
# Step 1: Construct the TF-IDF matrix for the entire document corpus
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_documents = tfidf_vectorizer.fit_transform(documents_tokenized_no_stopwords)

# Step 2: Select the top p stems based on their global importance (sum of TF-IDF scores across all documents)
p = 100  # Replace with desired value of p
tfidf_scores = tfidf_matrix_documents.sum(axis=0).A1  # Sum TF-IDF scores for each term across all documents
top_p_indices = np.argsort(tfidf_scores)[-p:]         # Get indices of top p terms

# Step 3: Create a vocabulary of the top p stems
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
top_p_stems = feature_names[top_p_indices]

# Step 4: Build new TF-IDF matrices for documents and queries using the top p stems only
# Initialize TfidfVectorizer with the reduced vocabulary (top p stems)
tfidf_vectorizer_top_p = TfidfVectorizer(vocabulary=top_p_stems)

# Construct the reduced TF-IDF matrix for documents
tfidf_matrix_documents_top_p = tfidf_vectorizer_top_p.fit_transform(documents)

# Construct the reduced TF-IDF matrix for queries using the same top p vocabulary
tfidf_matrix_queries_top_p = tfidf_vectorizer_top_p.transform(queries)

# Output: TF-IDF matrices for documents and queries using only the top p stems
print("Reduced TF-IDF matrix for documents:", tfidf_matrix_documents_top_p)
print("Reduced TF-IDF matrix for queries:", tfidf_matrix_queries_top_p)

Reduced TF-IDF matrix for documents: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 61706 stored elements and shape (6462, 100)>
  Coords	Values
  (0, 89)	0.3638299518471622
  (0, 81)	0.421762794889872
  (0, 68)	0.47387245665338407
  (0, 40)	0.46382772102430087
  (0, 36)	0.5000526482484273
  (1, 92)	0.20893987632630068
  (1, 89)	0.36475581935474544
  (1, 81)	0.2114180443945803
  (1, 74)	0.271573479820855
  (1, 66)	0.478704068075433
  (1, 60)	0.21477972463210213
  (1, 58)	0.30894930254027214
  (1, 40)	0.4650080609436299
  (1, 13)	0.3433329181745843
  (2, 93)	0.2831980019086068
  (2, 91)	0.26206441338703174
  (2, 83)	0.24797913806297958
  (2, 74)	0.7217642450248546
  (2, 20)	0.36825653212511533
  (2, 0)	0.3647889570525824
  (3, 81)	0.21367573475668397
  (3, 71)	0.228300858965186
  (3, 52)	0.5934740541812416
  (3, 47)	0.23899618860377814
  (3, 36)	0.25333936119094647
  :	:
  (6460, 80)	0.1877347240513384
  (6460, 78)	0.17263860401490902
  (6460, 72)	0.21723494143448396
  (6

In [22]:
# Step 1: Apply LSI using Truncated SVD on the document TF-IDF matrix
svd = TruncatedSVD(n_components=num_topics)
lsi_document_matrix = svd.fit_transform(tfidf_matrix_documents_top_p)

# Step 2: Transform the query TF-IDF matrix to the same LSI space
lsi_query_matrix = svd.transform(tfidf_matrix_queries_top_p)

# Step 3: Calculate cosine similarity between each query and each document in the LSI-reduced space
similarity_matrix = cosine_similarity(lsi_query_matrix, lsi_document_matrix)

# Step 4: Rank documents for each query based on similarity and display top 5 results
top_k = 5
for query_idx, similarities in enumerate(similarity_matrix):
    # Get indices of top 5 most similar documents
    top_doc_indices = np.argsort(similarities)[-top_k:][::-1]
    print(f"Query {query_idx + 1}:")
    for rank, doc_idx in enumerate(top_doc_indices, start=1):
        print(f"  Rank {rank}: Document {doc_idx + 1} (Score: {similarities[doc_idx]:.4f})")
    print("\n")

Query 1:
  Rank 1: Document 2238 (Score: 0.9813)
  Rank 2: Document 2389 (Score: 0.9154)
  Rank 3: Document 2478 (Score: 0.8951)
  Rank 4: Document 2750 (Score: 0.8787)
  Rank 5: Document 3449 (Score: 0.8522)


Query 2:
  Rank 1: Document 3636 (Score: 1.0000)
  Rank 2: Document 1730 (Score: 1.0000)
  Rank 3: Document 6048 (Score: 1.0000)
  Rank 4: Document 3491 (Score: 1.0000)
  Rank 5: Document 2958 (Score: 1.0000)


Query 3:
  Rank 1: Document 5930 (Score: 0.9446)
  Rank 2: Document 1619 (Score: 0.9235)
  Rank 3: Document 1874 (Score: 0.8770)
  Rank 4: Document 1422 (Score: 0.8661)
  Rank 5: Document 1371 (Score: 0.8235)


Query 4:
  Rank 1: Document 1600 (Score: 0.9294)
  Rank 2: Document 244 (Score: 0.9183)
  Rank 3: Document 1749 (Score: 0.9005)
  Rank 4: Document 541 (Score: 0.8929)
  Rank 5: Document 1232 (Score: 0.8007)


Query 5:
  Rank 1: Document 1545 (Score: 0.9366)
  Rank 2: Document 2549 (Score: 0.9274)
  Rank 3: Document 3169 (Score: 0.8743)
  Rank 4: Document 3798 (Scor

<h2>PROBABILISTIC MODEL</h2><br>
Here, we are basically calculating TF-IDF matrices of both documents and queries in the space of vocabulary created by the entire corpus of documents.<br>
We basically create 2 dictionaries where one dictionary contains the relevance score for each term, and other dictionary contains the non-relevance score for each term. Initially, relevance scores are taken as 0.5 (it signifies that the relevance of each word is uncertain) and non-relevance scores are taken to be # documents having the term/ total # documents (higher values of this signify that the word is not very relevant as it is present in every document, just like IDF).

In [33]:
from collections import defaultdict
import math

# Step 1: Build Vocabulary and Compute Initial TF-IDF for Documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_documents = tfidf_vectorizer.fit_transform(documents_tokenized)
tfidf_matrix_queries = tfidf_vectorizer.transform(queries_tokenized)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Initialize relevance probabilities and calculate non-relevance probabilities
vocabulary_size = len(feature_names)
relevance_prob = defaultdict(lambda: 0.5)
non_relevance_prob = defaultdict(float)

# Total number of documents in the corpus
total_documents = tfidf_matrix_documents.shape[0]

# Calculate document frequency for each term to determine non-relevance probability
document_frequency = np.diff(tfidf_matrix_documents.tocsc().indptr)

# Initial non-relevance probabilities
for term_idx, term in enumerate(feature_names):
    non_relevance_prob[term] = document_frequency[term_idx] / total_documents

# Step 2: Define function to calculate similarity with weighted log-odds
def calculate_similarity(query_idx):
    query_vector = tfidf_matrix_queries[query_idx]
    similarity_scores = []

    for doc_idx in range(tfidf_matrix_documents.shape[0]):
        doc_vector = tfidf_matrix_documents[doc_idx]
        score = 0

        # Iterate over terms common to both query and document
        for term_idx in set(query_vector.nonzero()[1]).intersection(doc_vector.nonzero()[1]):
            term = feature_names[term_idx]
            w_iq = query_vector[0, term_idx]
            w_id = doc_vector[0, term_idx]

            # Fetch relevance and non-relevance probabilities
            rel_prob = relevance_prob[term]
            non_rel_prob = non_relevance_prob[term]

            if 0 < rel_prob < 1 and 0 < non_rel_prob < 1:  # To avoid log(0) errors
                log_odds_relevance = math.log((1 - rel_prob) / rel_prob)
                log_odds_non_relevance = math.log((1 - non_rel_prob) / non_rel_prob)
                score += w_iq * w_id * (log_odds_relevance + log_odds_non_relevance)

        similarity_scores.append((doc_idx, score))
    
    # Sort documents for the query by similarity score
    return sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Step 3: Rank documents for each query and update probabilities
def rank_and_update(queries, V=10, top_k=5):
    results = {}

    for query_idx in range(len(queries)):
        print(f"Processing Query {query_idx + 1}: {queries[query_idx]}")
        
        # Calculate similarity scores and rank documents
        ranked_docs = calculate_similarity(query_idx)
        top_documents = ranked_docs[:top_k]

        # Display top 5 results
        print(f"Top {top_k} documents for query '{queries[query_idx]}':")
        for rank, (doc_idx, score) in enumerate(top_documents, start=1):
            print(f"  Rank {rank}: Document {doc_idx + 1} (Score: {score:.4f})")

        results[queries[query_idx]] = top_documents

        # Update probabilities using the top V documents
        subset_documents = ranked_docs[:V]  # Select top V documents for updating
        V = len(subset_documents)  # Redefine V based on selected documents

        for term_idx in range(len(feature_names)):
            term = feature_names[term_idx]
            n_i = document_frequency[term_idx]  # Total documents containing the term
            V_i = sum(1 for doc_idx, _ in subset_documents if tfidf_matrix_documents[doc_idx, term_idx] > 0)

            # Update relevance and non-relevance probabilities using the formulas
            relevance_prob[term] = (V_i + 0.5) / (V + 1)
            non_relevance_prob[term] = (n_i - V_i + 0.5) / (total_documents - V + 1)

    return results

# Run the ranking and iterative updating process
results = rank_and_update(queries, V=10, top_k=5)

Processing Query 1: What Is The Cost Of Disability Insurance For Individuals? 
Top 5 documents for query 'What Is The Cost Of Disability Insurance For Individuals? ':
  Rank 1: Document 3205 (Score: 1.0856)
  Rank 2: Document 2274 (Score: 0.9999)
  Rank 3: Document 2408 (Score: 0.9576)
  Rank 4: Document 2020 (Score: 0.8924)
  Rank 5: Document 2406 (Score: 0.8904)
Processing Query 2: Does Life Insurance Count In Net Worth? 
Top 5 documents for query 'Does Life Insurance Count In Net Worth? ':
  Rank 1: Document 970 (Score: 4.3575)
  Rank 2: Document 972 (Score: 3.9443)
  Rank 3: Document 2236 (Score: 3.5241)
  Rank 4: Document 4121 (Score: 2.5263)
  Rank 5: Document 2234 (Score: 2.4332)
Processing Query 3: Can People With Diabetes Get Life Insurance? 
Top 5 documents for query 'Can People With Diabetes Get Life Insurance? ':
  Rank 1: Document 4103 (Score: 3.0753)
  Rank 2: Document 3904 (Score: 3.0702)
  Rank 3: Document 3905 (Score: 3.0271)
  Rank 4: Document 5073 (Score: 3.0264)
  R

Here, we are just calculating the same thing but using documents and queries where stop words have been removed.

In [6]:
from collections import defaultdict
import math
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Build Vocabulary and Compute Initial TF-IDF for Documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_documents = tfidf_vectorizer.fit_transform(documents_tokenized_no_stopwords)
tfidf_matrix_queries = tfidf_vectorizer.transform(queries_tokenized_no_stopwords)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Initialize relevance probabilities and calculate non-relevance probabilities
vocabulary_size = len(feature_names)
relevance_prob = defaultdict(lambda: 0.5)
non_relevance_prob = defaultdict(float)

# Total number of documents in the corpus
total_documents = tfidf_matrix_documents.shape[0]

# Calculate document frequency for each term to determine non-relevance probability
document_frequency = np.diff(tfidf_matrix_documents.tocsc().indptr)

# Initial non-relevance probabilities
for term_idx, term in enumerate(feature_names):
    non_relevance_prob[term] = document_frequency[term_idx] / total_documents

# Step 2: Define function to calculate similarity with weighted log-odds
def calculate_similarity(query_idx):
    query_vector = tfidf_matrix_queries[query_idx]
    similarity_scores = []

    for doc_idx in range(tfidf_matrix_documents.shape[0]):
        doc_vector = tfidf_matrix_documents[doc_idx]
        score = 0

        # Iterate over terms common to both query and document
        for term_idx in set(query_vector.nonzero()[1]).intersection(doc_vector.nonzero()[1]):
            term = feature_names[term_idx]
            w_iq = query_vector[0, term_idx]
            w_id = doc_vector[0, term_idx]

            # Fetch relevance and non-relevance probabilities
            rel_prob = relevance_prob[term]
            non_rel_prob = non_relevance_prob[term]

            if 0 < rel_prob < 1 and 0 < non_rel_prob < 1:  # To avoid log(0) errors
                log_odds_relevance = math.log((1 - rel_prob) / rel_prob)
                log_odds_non_relevance = math.log((1 - non_rel_prob) / non_rel_prob)
                score += w_iq * w_id * (log_odds_relevance + log_odds_non_relevance)

        similarity_scores.append((doc_idx, score))
    
    # Sort documents for the query by similarity score
    return sorted(similarity_scores, key=lambda x: x[1], reverse=True)

# Step 3: Rank documents for each query and update probabilities
def rank_and_update(queries, V=10, top_k=5):
    results = {}

    for query_idx in range(len(queries)):
        print(f"Processing Query {query_idx + 1}: {queries[query_idx]}")
        
        # Calculate similarity scores and rank documents
        ranked_docs = calculate_similarity(query_idx)
        top_documents = ranked_docs[:top_k]

        # Display top 5 results
        print(f"Top {top_k} documents for query '{queries[query_idx]}':")
        for rank, (doc_idx, score) in enumerate(top_documents, start=1):
            print(f"  Rank {rank}: Document {doc_idx + 1} (Score: {score:.4f})")

        results[queries[query_idx]] = top_documents

        # Update probabilities using the top V documents
        subset_documents = ranked_docs[:V]  # Select top V documents for updating
        V = len(subset_documents)  # Redefine V based on selected documents

        for term_idx in range(len(feature_names)):
            term = feature_names[term_idx]
            n_i = document_frequency[term_idx]  # Total documents containing the term
            V_i = sum(1 for doc_idx, _ in subset_documents if tfidf_matrix_documents[doc_idx, term_idx] > 0)

            # Update relevance and non-relevance probabilities using the formulas
            relevance_prob[term] = (V_i + 0.5) / (V + 1)
            non_relevance_prob[term] = (n_i - V_i + 0.5) / (total_documents - V + 1)

    return results

# Run the ranking and iterative updating process
results = rank_and_update(queries, V=10, top_k=5)

Processing Query 1: What Is The Cost Of Disability Insurance For Individuals? 
Top 5 documents for query 'What Is The Cost Of Disability Insurance For Individuals? ':
  Rank 1: Document 3205 (Score: 1.4397)
  Rank 2: Document 2274 (Score: 1.3214)
  Rank 3: Document 2020 (Score: 1.2944)
  Rank 4: Document 2408 (Score: 1.2496)
  Rank 5: Document 6129 (Score: 1.1718)
Processing Query 2: Does Life Insurance Count In Net Worth? 
Top 5 documents for query 'Does Life Insurance Count In Net Worth? ':
  Rank 1: Document 970 (Score: 4.8764)
  Rank 2: Document 2236 (Score: 4.6324)
  Rank 3: Document 972 (Score: 4.5779)
  Rank 4: Document 2234 (Score: 3.0242)
  Rank 5: Document 4121 (Score: 2.9464)
Processing Query 3: Can People With Diabetes Get Life Insurance? 
Top 5 documents for query 'Can People With Diabetes Get Life Insurance? ':
  Rank 1: Document 3904 (Score: 3.6300)
  Rank 2: Document 1767 (Score: 3.5739)
  Rank 3: Document 3905 (Score: 3.5040)
  Rank 4: Document 68 (Score: 3.4379)
  Ran